# Puebas
Me paso del org-mode a jupyter-notebook, porque es más cómodo, y esta más extendido su uso, para probar el código. Para realizar el TFG seguramente sí use org -mode para generar el documento en $\LaTeX$.

Por ahora no he hecho nada, sólo estoy probando el código que ya había escrito (y que tengo que reacer). Una ventaja es que se pueden visualizar más o menos correctamente los churros de matrices que salen.

In [1]:
from sympy import *
import pandas as pd
a=Symbol('a', real=True, positive=True)
q_x=Symbol('q_x', real=True);  q_y=Symbol('q_y', real=True)
q=Matrix([q_x,q_y])
a_1=Matrix([a,0]); a_2=Rational(1,2)*Matrix([-a,sqrt(3)*a])
R_B=Rational(1,3)*a_1+Rational(2,3)*a_2; R_N=Rational(2,3)*a_1+Rational(1,3)*a_2

alpha_1=Symbol('alpha_1') #constante de fuerza entre 1eros vecinos
alpha_2=Symbol('alpha_2') #constante de fuerza entre 2ndos vecinos
alpha_3=Symbol('alpha_3') #constante de fuerza entre 3eros vecinos
alpha_4=Symbol('alpha_4') #constante de fuerza entre 4tos  vecinos

m_1, m_2, omega=symbols("m_1, m_2, omega") #masa de los átomos de Boro y N.
A_1_x, A_1_y, A_1_z, A_2_x, A_2_y, A_2_z = symbols("A_1_x, A_1_y, A_1_z, A_2_x, A_2_y, A_2_z")
A=Matrix([A_1_x,A_1_y,A_1_z,A_2_x,A_2_y,A_2_z])

def A_plano(nu):
    if nu == 1:
        return Matrix([A_1_x,A_1_y])

    elif nu == 2:
        return Matrix([A_2_x,A_2_y])

    else:
        print("Error, nu sólo puede se 1 o 2")

def A_perp_plano(nu):
    if nu == 1:
        return A_1_z

    elif nu == 2:
        return A_2_z

    else:
        print("Error, nu sólo puede se 1 o 2")


def R_l(l_1,l_2):
    return l_1*a_1+l_2*a_2

def R_nu_l(nu,l_1,l_2):
    if nu == 1:
        return l_1*a_1+l_2*a_2+R_B

    elif nu == 2:
        return l_1*a_1+l_2*a_2+R_N

    else:
        print("Error, nu solo puede ser 1 o 2 ")

def R_hat(nuprima,nu,l_1,l_2):
    if (R_nu_l(nu,l_1,l_2)-R_nu_l(nuprima,0,0)).norm()>0:
        return (R_nu_l(nu,l_1,l_2)-R_nu_l(nuprima,0,0))/(R_nu_l(nu,l_1,l_2)-R_nu_l(nuprima,0,0)).norm()

    else:
        return (R_nu_l(nu,l_1,l_2)-R_nu_l(nuprima,0,0))

def propiedades_atomos(l_1, l_2):
    return [ (k, m, i, j, R_hat(k,m,i,j)*R_hat(k,m,i,j).T*
              (A_plano(m)*exp(I*q.dot(R_l(i,j)))-A_plano(k)),
              Matrix([A_perp_plano(m)*exp(I*q.dot(R_l(i,j))) 
              -A_perp_plano(k)]), (R_nu_l(m,i,j)-R_nu_l(k,0,0)).norm()/a)
              for k in [1,2] for m in [1,2] for i in range(-l_1,l_1+1) for j in range(-l_2,l_2+1)]

columnas = [r"$\nu\prime$",r"$\nu",r"$l_1$", r"$l_2$",'Ecuación','Ecuación perp.','Distancia']

def Atomos(l_1, l_2):
    return pd.DataFrame(propiedades_atomos(l_1, l_2),columns=columnas).sort_values(['Distancia',r"$\nu\prime$"], ascending=[True, True])

PrimerosVecinosBoro= Atomos(1,1)[(Atomos(1,1)['Distancia']<0.9) & (Atomos(1,1)['Distancia']>0) & (Atomos(1,1)[r"$\nu\prime$"]==1)]

PrimerosVecinosNitrogeno= Atomos(1,1)[(Atomos(1,1)['Distancia']<0.9) & (Atomos(1,1)['Distancia']>0) & (Atomos(1,1)[r"$\nu\prime$"]==2)]

SegundosVecinosBoro= Atomos(1,1)[(Atomos(1,1)['Distancia']>0.9) & (Atomos(1,1)['Distancia']<1.1) & (Atomos(1,1)[r"$\nu\prime$"]==1)]

SegundosVecinosNitrogeno= Atomos(1,1)[(Atomos(1,1)['Distancia']>0.9) & (Atomos(1,1)['Distancia']<1.1) & (Atomos(1,1)[r"$\nu\prime$"]==2)]

TercerosVecinosBoro= Atomos(1,1)[(Atomos(1,1)['Distancia']>1.1) & (Atomos(1,1)['Distancia']<1.5) & (Atomos(1,1)[r"$\nu\prime$"]==1)]

TercerosVecinosNitrogeno= Atomos(1,1)[(Atomos(1,1)['Distancia']> 1.1) &  (Atomos(1,1)['Distancia']<1.5) & (Atomos(1,1)[r"$\nu\prime$"]==2)]

CuartosVecinosBoro= Atomos(2,2)[(Atomos(2,2)['Distancia']>1.5) & (Atomos(2,2)['Distancia']<1.7) & (Atomos(2,2)[r"$\nu\prime$"]==1)]

CuartosVecinosNitrogeno= Atomos(2,2)[(Atomos(2,2)['Distancia']> 1.5) & (Atomos(2,2)['Distancia']<1.7) & (Atomos(2,2)[r"$\nu\prime$"]==2)]

In [2]:
M1=PrimerosVecinosBoro['Ecuación'].sum().row_insert(2, 
                      PrimerosVecinosBoro['Ecuación perp.'].sum()).row_insert(3, 
                      PrimerosVecinosNitrogeno['Ecuación'].sum()).row_insert(5, 
                      PrimerosVecinosNitrogeno['Ecuación perp.'].sum())
Mat1=Matrix([M1[i].coeff(j) for i in range(0,6) for j in 
             [A_1_x, A_1_y, A_1_z, A_2_x, A_2_y,A_2_z]]).reshape(6,6)

M2=SegundosVecinosBoro['Ecuación'].sum().row_insert(2, 
                      SegundosVecinosBoro['Ecuación perp.'].sum()).row_insert(3, 
                      SegundosVecinosNitrogeno['Ecuación'].sum()).row_insert(5, 
                      SegundosVecinosNitrogeno['Ecuación perp.'].sum())

Mat2=Matrix([M2[i].coeff(j) for i in range(0,6) for j in 
             [A_1_x, A_1_y, A_1_z, A_2_x, A_2_y,A_2_z]]).reshape(6,6)

M3=TercerosVecinosBoro['Ecuación'].sum().row_insert(2, 
                      TercerosVecinosBoro['Ecuación perp.'].sum()).row_insert(3, 
                      TercerosVecinosNitrogeno['Ecuación'].sum()).row_insert(5, 
                      TercerosVecinosNitrogeno['Ecuación perp.'].sum())

Mat3=Matrix([M3[i].coeff(j) for i in range(0,6) for j in 
        [A_1_x, A_1_y, A_1_z, A_2_x, A_2_y,A_2_z]]).reshape(6,6)

M4=CuartosVecinosBoro['Ecuación'].sum().row_insert(2, 
                      CuartosVecinosBoro['Ecuación perp.'].sum()).row_insert(3, 
                      CuartosVecinosNitrogeno['Ecuación'].sum()).row_insert(5, 
                      CuartosVecinosNitrogeno['Ecuación perp.'].sum())
Mat4=Matrix([M4[i].coeff(j) for i in range(0,6) for j in 
        [A_1_x, A_1_y, A_1_z, A_2_x, A_2_y,A_2_z]]).reshape(6,6)

In [3]:
Mat1

Matrix([
[                  -3/2,                                                         0,                                                     0, 3/4 + 3*exp(-I*a*q_x)/4,                                                           0,                                                       0],
[                     0,                                                      -3/2,                                                     0,                       0, exp(I*(-a*q_x/2 + sqrt(3)*a*q_y/2)) + 1/4 + exp(-I*a*q_x)/4,                                                       0],
[                     0,                                                         0,                                                    -3,                       0,                                                           0, exp(I*(-a*q_x/2 + sqrt(3)*a*q_y/2)) + 1 + exp(-I*a*q_x)],
[3*exp(I*a*q_x)/4 + 3/4,                                                         0,                                                     0,    

In [4]:
Mat2

Matrix([
[exp(I*(-a*q_x/2 - sqrt(3)*a*q_y/2))/4 + exp(I*(-a*q_x/2 + sqrt(3)*a*q_y/2))/4 + exp(I*(a*q_x/2 - sqrt(3)*a*q_y/2))/4 + exp(I*(a*q_x/2 + sqrt(3)*a*q_y/2))/4 + exp(I*a*q_x) - 3 + exp(-I*a*q_x),                                                                                                                                                                       0,                                                                                                                                                                                      0,                                                                                                                                                                                              0,                                                                                                                                                                       0,                                                                                     

In [5]:
Mat3

Matrix([
[                                                                              -3/2,                                                                                                                   0,                                                                                                               0, 3*exp(I*(-3*a*q_x/2 + sqrt(3)*a*q_y/2))/4 + 3*exp(I*(a*q_x/2 + sqrt(3)*a*q_y/2))/4,                                                                                                                    0,                                                                                                                0],
[                                                                                 0,                                                                                                                -3/2,                                                                                                               0,                                             

In [6]:
Mat4

Matrix([
[                                                                                                                                                                                                -3,                                                                                                                                                                                              0,                                                                                                                                                                    0, 3*exp(I*(-3*a*q_x/2 - sqrt(3)*a*q_y/2))/7 + 3*exp(I*(-a*q_x + sqrt(3)*a*q_y))/28 + 3*exp(I*(a*q_x/2 - sqrt(3)*a*q_y/2))/7 + 27*exp(I*a*q_x)/28 + 3*exp(sqrt(3)*I*a*q_y)/28 + 27*exp(-2*I*a*q_x)/28,                                                                                                                                                                                              0,                                                 

In [7]:
Mat1+Mat2+Mat3+Mat4

Matrix([
[                                                                                                                  exp(I*(-a*q_x/2 - sqrt(3)*a*q_y/2))/4 + exp(I*(-a*q_x/2 + sqrt(3)*a*q_y/2))/4 + exp(I*(a*q_x/2 - sqrt(3)*a*q_y/2))/4 + exp(I*(a*q_x/2 + sqrt(3)*a*q_y/2))/4 + exp(I*a*q_x) - 9 + exp(-I*a*q_x),                                                                                                                                                                                                                                                                                                                                                                                0,                                                                                                                                                                                                                                                                                                                           